In [1]:
cd ..

c:\Users\14102\Brown\Realizing_Rights\RealizingRights


In [2]:
from utilities.web_assist import recurse_scan_all_unique_links_in_site, closest_link_match, make_https, prepend_root_to_url, make_driver_utils
from url_scraper.link_data import LinkData
import pandas as pd

In [3]:
district_df = pd.read_excel(io='data/USSchoolDistrictWebsiteInfo.xlsx', sheet_name='ELSI Export', header=6)
district_df['URL'] = district_df['URL'].apply(lambda x: make_https(url=x))

district_df['NumStudents'] = pd.to_numeric(district_df['NumStudents'], errors='coerce')
district_df_with_urls = district_df[~district_df['URL'].isna()]
district_df_with_urls['URL'] = district_df_with_urls['URL'].apply(lambda x: x.lower())
district_df_with_urls['URL'] = district_df_with_urls['URL'].apply(lambda x: prepend_root_to_url(base_url=x, prefix=''))
district_df_with_urls = district_df_with_urls[district_df_with_urls['NumStudents'] > 0]
district_df_with_urls = district_df_with_urls[district_df_with_urls['NumSchools'] > 0].reset_index(drop=True)
district_df_with_urls.sort_values(by='NumStudents', ascending=False, inplace=True)

c:\Users\14102\anaconda3\envs\real_right_env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [ ]:
drvr, actions, wait = make_driver_utils()

blacklist_terms = ['login', '.aspx', 'lightbox', 'file', '.php', '#', '.pdf', 'doc', 'jpg', 'tel:']

link_keywords = ['boe meeting', 'board meeting', 'board of education', 'parent', 'school board', 'board minutes',
                 'meeting', 'video recording', 'board video recording', 'superintendent', 'document']

board_meeting_keywords = ['boe meeting', 'board meeting', 'boe recording', 'board recording', 'minutes'
                            'boe meeting recording', 'board meeting recording', 'boe meeting video',
                            'board meeting video', 'boe video recording', 'board video recording']
url_data = {}
for _, row in district_df_with_urls[:50].iterrows():

    url = row['URL']
    start_link_set = set()
    dist_info = []
    start_link_set.add(LinkData(link_text='BASE', link_url=url, depth=0))
    recursed_lcl_links, recursed_ext_links = set(), set()
    recursed_lcl_links, recursed_ext_links = recurse_scan_all_unique_links_in_site(url=url,
                                                                                    base_url=url, 
                                                                                    local_link_set=start_link_set,
                                                                                    external_link_set=set(),
                                                                                    drvr=drvr, 
                                                                                    actions=actions,
                                                                                    blacklist_terms=blacklist_terms,
                                                                                    wait=wait,
                                                                                    link_keywords=link_keywords)

    cur_sim = 50
    best_link = None
    lcl_links_sorted = sorted(list(recursed_lcl_links), key=lambda x: x.depth_found)
    ext_links_sorted = sorted(list(recursed_ext_links), key=lambda x: x.depth_found)
    for lcl_link in lcl_links_sorted:
        if cur_sim == 100:
            break
        sim = closest_link_match(lcl_link.link_text, board_meeting_keywords)
        if sim > cur_sim:
            best_link = lcl_link
            cur_sim = sim

    for ext_link in ext_links_sorted:
        if cur_sim == 100:
            break
        if ext_link.link_text:
            sim = closest_link_match(ext_link.link_text, board_meeting_keywords)
            if sim > cur_sim:
                best_link = ext_link
                cur_sim = sim

    # Identify External Links Pointing to social media sites 
    sites_identified = {}
    num_ext_links = len(ext_links_sorted)
    ext_id = 0
    social_media_sites = ['youtube', 'vimeo', 'facebook', 'twitter']
    while ext_id < num_ext_links:
        ext_link = ext_links_sorted[ext_id]
        scl_id = 0
        while scl_id < len(social_media_sites):
            if social_media_sites[scl_id] in ext_link.link_url:
                sites_identified[social_media_sites.pop(scl_id)] = ext_link.depth_found
            scl_id += 1

        ext_id += 1

        if len(social_media_sites) == 0:
            break
    
    if best_link:
        url_data[row['Agency ID']] = (sites_identified, best_link.link_text, best_link.link_url, best_link.depth_found)
    else: 
        url_data[row['Agency ID']] = (sites_identified, None, None, None)
    print(sites_identified, best_link)



In [ ]:
df = pd.DataFrame(columns=['District ID', 'brd_mtn_URL_text', 'brd_mtn_URL_Link', 'brd_mtn_URL_depth', 'facebook', 'youtube', 'twitter', 'vimeo'])


for url, data in url_data.items():
    row = {
        'District ID': url,
        'brd_mtn_URL_text': data[1],
        'brd_mtn_URL_Link': data[2],
        'brd_mtn_URL_depth': data[3],
        'facebook': None,
        'youtube': None,
        'twitter': None,
        'vimeo': None
    }

    if data[0]:
        row.update(data[0])

    df = df.append(row, ignore_index=True)

df

In [ ]:
scanned_dist_df = df.merge(district_df_with_urls, left_on='District ID', right_on='Agency ID')
scanned_dist_df.to_csv('data/SampleOutput.csv')

In [8]:
district_df = pd.read_excel(io='data/USSchoolDistrictWebsiteInfo.xlsx', sheet_name='ELSI Export', header=6)
district_df['NumStudents'] = pd.to_numeric(district_df['NumStudents'], errors='coerce')
district_df.sort_values(by='NumStudents', ascending=False).head(50)

,Agency Name,State Name,State Abbr,Agency ID,NumSchools,LowGrade,HighGrade,NumStudents,URL
10035,LOS ANGELES UNIFIED,California,CA,622710.0,778.0,Kindergarten,12th Grade,435958.0,http://www.lausd.net
3339,CITY OF CHICAGO SD 299,Illinois,IL,1709930.0,646.0,Prekindergarten,12th Grade,329836.0,http://www.cps.k12.il.us
10965,MIAMI-DADE,Florida,FL,1200390.0,522.0,Prekindergarten,12th Grade,328589.0,http://www.dadeschools.net
3376,CLARK COUNTY SCHOOL DISTRICT,Nevada,NV,3200060.0,380.0,Prekindergarten,12th Grade,315787.0,NaN
14193,PUERTO RICO DEPARTMENT OF EDUCATION,Puerto Rico,PR,7200030.0,851.0,Prekindergarten,12th Grade,259535.0,http://www.de.pr.gov
2151,BROWARD,Florida,FL,1200180.0,332.0,Prekindergarten,12th Grade,256037.0,http://www.browardschools.com
7728,HILLSBOROUGH,Florida,FL,1200870.0,304.0,Prekindergarten,12th Grade,224149.0,http://www.sdhc.k12.fl.us/
12971,ORANGE,Florida,FL,1201440.0,274.0,Prekindergarten,12th Grade,203224.0,http://www.ocps.net
7946,HOUSTON ISD,Texas,TX,4823640.0,274.0,Prekindergarten,12th Grade,194607.0,http://www.houstonisd.org
13198,PALM BEACH,Florida,FL,1201500.0,235.0,Prekindergarten,12th Grade,187943.0,http://www.palmbeachschools.org
